In [10]:
import os

In [11]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [12]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH12-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH12-RAG


In [13]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [14]:
# 단계 1: 문서 로드(Load Documents)
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
url = "https://n.news.naver.com/article/437/0000378416"
loader = WebBaseLoader(
    web_paths=(url,),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)
docs = loader.load()


# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

splits = text_splitter.split_documents(docs)

# 단계 3: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 단계 4: 검색(Search)
# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 5: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = hub.pull("rlm/rag-prompt")

# 단계 6: 언어모델 생성(Create LLM)
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


# 단계 7: 체인 생성(Create Chain)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "부영그룹의 출산 장려 정책에 대해 설명해주세요"
response = rag_chain.invoke(question)

# 결과 출력
print(f"URL: {url}")
print(f"문서의 수: {len(docs)}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

URL: https://n.news.naver.com/article/437/0000378416
문서의 수: 1
[HUMAN]
부영그룹의 출산 장려 정책에 대해 설명해주세요

[AI]
부영그룹은 출산을 장려하기 위해 2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다. 해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다. 또한, 셋째까지 낳는 경우에는 국민주택을 제공할 예정입니다.


## 단계 1: 문서 로드(Load Documents)

In [15]:
# 뉴스기사의 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
loader = WebBaseLoader(
    web_paths=("https://www.bbc.com/news/business-68092814",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "main",
            attrs={"id": ["main-content"]},
        )
    ),
)
docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs[0].page_content[:500]

문서의 수: 1


'Could AI \'trading bots\' transform the world of investing?Getty ImagesIt is hard for both humans and computers to predict stock market movementsSearch for "AI investing" online, and you\'ll be flooded with endless offers to let artificial intelligence manage your money.I recently spent half an hour finding out what so-called AI "trading bots" could apparently do with my investments.Many prominently suggest that they can give me lucrative returns. Yet as every reputable financial firm warns - your '

### PDF

In [16]:
# !pip install pypdf


In [19]:
# from langchain.document_loaders import PyPDFLoader

# # PDF 파일 로드. 파일의 경로 입력
# loader = PyPDFLoader("data/1568)누구나 한번쯤 읽어야 할 사서삼경 (미리내공방) .pdf")

# # 페이지 별 문서 로드
# docs = loader.load()
# print(f"문서의 수: {len(docs)}")

# # 10번째 페이지의 내용 출력
# print(f"\n[페이지내용]\n{docs[10].page_content[:500]}")
# print(f"\n[metadata]\n{docs[10].metadata}\n")

In [20]:
import fitz  # PyMuPDF
from langchain_core.document_loaders.base import Document

class CustomPDFLoader:
    def __init__(self, file_path: str):
        self.file_path = file_path

    def load(self):
        documents = []
        with fitz.open(self.file_path) as doc:
            for page in doc:
                text = page.get_text()
                documents.append(Document(page_content=text))
        return documents

# PDF 파일 로드
loader = CustomPDFLoader("data/1568)누구나 한번쯤 읽어야 할 사서삼경 (미리내공방) .pdf")

# 페이지 별 문서 로드
docs = loader.load()

# 문서의 수 출력
print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
if len(docs) >= 10:
    print(docs[9].page_content)


문서의 수: 353
중용이란?  …208
궁색해도 개천이 될 수는 없어 214 | 돈을 주고 산 어리석음 216 | 기수는 말과 한
마음이 되어야 219 | 현명한 아내 덕에 출세한 사내 221 | 간사한 지혜 224 | 치욕
을 안겨준 은인 226 | 어려워도 꿋꿋한 것이 용기 230 | 인생의 정곡을 짚을 줄 알
아야 234 | 계획보다 중요한 실천 237 | 물의 도에 몸을 맡긴 사내 239 | 《중용》의 
명구절 242
서경이란?  …258
백성의 마음은 하늘의 마음 262 | 백성에게 다가갈 수 있어야 264 | 도둑을 없애
려면 268 | 장단점을 받아들이는 귀 271 | 정신이상자들이 사는 세상 273 | 유비
무환 276 | 그릇 속의 백성들 278 | 《서경》의 명구절 280
주역이란?  …292
벼랑 끝에 선 명궁 298 | 선비의 헛된 죽음 300 | 징후가 보이면 결단을 304 | 《주
역》의 명구절 306 
시경이란?  …316
나팔꽃 사랑 320 | 문둥병을 고친 아내의 사랑 324 | 폭군에게 간한 충신 327 | 《시
경》의 명구절 330 
中
庸
書
經
周
易
詩
經



### CSV

In [21]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# CSV 파일 로드
loader = CSVLoader(file_path="data/titanic.csv")
docs = loader.load()
print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[10].page_content[:500]}")
print(f"\n[metadata]\n{docs[10].metadata}\n")

문서의 수: 891

[페이지내용]
PassengerId: 11
Survived: 1
Pclass: 3
Name: Sandstrom, Miss. Marguerite Rut
Sex: female
Age: 4
SibSp: 1
Parch: 1
Ticket: PP 9549
Fare: 16.7
Cabin: G6
Embarked: S

[metadata]
{'source': 'data/titanic.csv', 'row': 10}



### TXT 파일

In [22]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/appendix-keywords.txt")
docs = loader.load()
print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[0].page_content[:500]}")
print(f"\n[metadata]\n{docs[0].metadata}\n")

문서의 수: 1

[페이지내용]
Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.
예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.
연관키워드: 토큰화, 자연어

[metadata]
{'source': 'data/appendix-keywords.txt'}



### 폴더 내의 모든 파일 로드

In [23]:
# !pip install unstructured


In [24]:
# !pip install python-magic-bin


##### 해당 폴더 내 모든 txt파일 로드

In [25]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(".", glob="data/*.txt", show_progress=True)
docs = loader.load()

print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[0].page_content[:500]}")
print(f"\n[metadata]\n{docs[0].metadata}\n")

  0%|          | 0/7 [00:00<?, ?it/s]

c:\Users\anHye\anaconda3\envs\rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 7/7 [00:09<00:00,  1.29s/it]

문서의 수: 7

[페이지내용]
user_api = sk-None-VbfIkf2bFeBqTEZv7XIVT3BlbkFJbeD4NmH47w7YZDXjtoqD

project_api = sk-proj-OvEjNPodwtzRSIUnOOc4T3BlbkFJ0tZAd2g3zubzMAKwuMYK

langchain_api = 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

[metadata]
{'source': 'data\\api_key.txt'}



##### 해당 폴더 내 모든 pdf 파일 로드

In [26]:
# !pip install "unstructured[pdf]"


In [27]:
import os
import fitz  # PyMuPDF
from langchain_core.document_loaders.base import Document

class CustomPDFLoader:
    def __init__(self, file_path: str):
        self.file_path = file_path

    def load(self):
        documents = []
        with fitz.open(self.file_path) as doc:
            for page in doc:
                text = page.get_text()
                documents.append(Document(page_content=text))
        return documents

# 특정 디렉토리에서 모든 PDF 파일 로드
directory_path = "data"
documents = []

for filename in os.listdir(directory_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(directory_path, filename)
        loader = CustomPDFLoader(file_path)
        documents.extend(loader.load())

# 문서의 수 출력
print(f"문서의 수: {len(documents)}\n")

# 각 문서의 내용을 출력
for doc in documents:
    print(doc.page_content)


문서의 수: 376


누구나 한번쯤 읽어야 할
사서삼경


삶을 일깨우는 
고전산책 시리즈
06
四書三經


5
흔히 ‘사서삼경四書三經’을 유학의 지침서라고 한다. 분명 틀리지 
않은 말이다. 그러나 사서삼경을 온전히 논하는 데에서 아무래도 
좀 부족한 정의이지 싶다. 유학 영역으로만 한정하기에는 사서삼경
의 품이 너무 넓기 때문이다. 실상 사서삼경은 유학의 지침서를 넘
어 동양인의 지침서, 더 나아가 인간의 지침서이다. 
알다시피 사서삼경은 7권의 책을 한데 묶어 일컫는 것으로, 사서
는 《논어論語》
·
《맹자孟子》
·
《중용中庸》
·
《대학大學》을, 삼경은 《시경詩
經》
·
《서경
書經》
·
《역경
易經(
주역周易)》을 의미한다. 여기에는 인
仁과 예
禮, 
효孝와 충절, 믿음과 우애, 지식과 탐구 등 사람이 살아가면서 알아
두어야 할 것이 총망라되어 있다. 그런 만큼 사서삼경은 인간의 정
머리말

6
신적 양식서라고 해도 과언이 아니겠다. 
《맹자》에 이런 구절이 있다. 
‘사람이 좋은 옷을 입고 배불리 먹으며 따뜻한 곳에서 잠만 잔다
면 개돼지와 뭐가 다르겠는가?’ 
살아가는 데 무엇보다 중요한 것이 의식주다. 하지만 그것만 가
지고는 사람답게 살 수 없다. 정신이 가난하면 아무리 풍족한 의식
주를 누릴지라도 사람들에게 인정받지 못한다. 맹자는 바로 그 점
을 꿰뚫어 보고 사람들에게 경고한 것이다. 
오늘날에도 사서삼경은 우리의 정신을 풍요롭게 살찌우는 데 더
없이 좋은 경전이다. 하지만 우리 대다수는 그 사실을 알면서도 그
냥 지나치거나 접할 기회를 다음으로 미루고 있는 실정이다. 이러
한 양상은 한자를 기반으로 하기에 난해하고 고루할 것이라는 막연
한 선입견도 한몫하고 있다.
‘과연 어떻게 하면 사서삼경을 좀 더 많은 이에게 인생의 지침서
로서 쉽게 전할 수 있을까?’
이 책은 바로 이 지점에서 출발했다. 이 책은 사서삼경이라는 
7권이 어떤 경전이며, 무슨 내용을 담고 있고, 무엇을 강조하는지 
총 7장에 걸쳐 쉬운 설명으로 일목요연하게 보여줄 

##### Python

In [28]:
from langchain_community.document_loaders import PythonLoader

loader = DirectoryLoader(".", glob="**/*.py", loader_cls=PythonLoader)
docs = loader.load()

print(f"문서의 수: {len(docs)}\n")
print("[메타데이터]\n")
print(docs[0].metadata)
print("\n========= [앞부분] 미리보기 =========\n")
print(docs[0].page_content[:500])

문서의 수: 1

[메타데이터]

{'source': 'data\\audio_utils.py'}

========= [앞부분] 미리보기 =========

import re
import os
from pytube import YouTube
from moviepy.editor import AudioFileClip, VideoFileClip
from pydub import AudioSegment
from pydub.silence import detect_nonsilent


def extract_abr(abr):
    youtube_audio_pattern = re.compile(r"\d+")
    kbps = youtube_audio_pattern.search(abr)
    if kbps:
        kbps = kbps.group()
        return int(kbps)
    else:
        return 0


def get_audio_filepath(filename):
    # audio 폴더가 없으면 생성
    if not os.path.isdir("audio"):
        os.mkdir("au


### 단계 2: 문서 분할(Split Documents)

In [29]:
# 뉴스기사의 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
loader = WebBaseLoader(
    web_paths=("https://www.bbc.com/news/business-68092814",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "main",
            attrs={"id": ["main-content"]},
        )
    ),
)
docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs[0].page_content[:500]

문서의 수: 1


'Could AI \'trading bots\' transform the world of investing?Getty ImagesIt is hard for both humans and computers to predict stock market movementsSearch for "AI investing" online, and you\'ll be flooded with endless offers to let artificial intelligence manage your money.I recently spent half an hour finding out what so-called AI "trading bots" could apparently do with my investments.Many prominently suggest that they can give me lucrative returns. Yet as every reputable financial firm warns - your '

##### CharacterTextSplitter

##### CharacterTextSplitter 클래스는 텍스트를 특정 크기의 청크로 분할하는 기능을 제공합니다.

- separator 매개변수는 청크를 구분하는 데 사용되는 문자열을 지정하며, 여기서는 두 개의 개행 문자("\n\n")를 사용합니다
- chunk_size는 각 청크의 최대 길이를 결정합니다
- chunk_overlap은 인접한 청크 간에 겹치는 문자의 수를 지정합니다.
- length_function은 청크의 길이를 계산하는 데 사용되는 함수를 결정하며, 기본적으로 문자열의 길이를 반환하는 len 함수가 사용됩니다.
- is_separator_regex는 separator가 정규 표현식으로 해석될지 여부를 결정하는 불리언 값입니다.

In [30]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

In [31]:
# chain of density 논문의 일부 내용을 불러옵니다
with open("data/chain-of-density.txt", "r") as f:
    text = f.read()[:500]

In [32]:
text_splitter = CharacterTextSplitter(
    chunk_size=100, chunk_overlap=10, separator="\n\n"
)
text_splitter.split_text(text)

['Selecting the “right” amount of information to include in a summary is a difficult task. \nA good summary should be detailed and entity-centric without being overly dense and hard to follow. To better understand this tradeoff, we solicit increasingly dense GPT-4 summaries with what we refer to as a “Chain of Density” (CoD) prompt. Specifically, GPT-4 generates an initial entity-sparse summary before iteratively incorporating missing salient entities without increasing the length. Summaries genera']

In [33]:
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10, separator="\n")
text_splitter.split_text(text)

['Selecting the “right” amount of information to include in a summary is a difficult task.',
 'A good summary should be detailed and entity-centric without being overly dense and hard to follow. To better understand this tradeoff, we solicit increasingly dense GPT-4 summaries with what we refer to as a “Chain of Density” (CoD) prompt. Specifically, GPT-4 generates an initial entity-sparse summary before iteratively incorporating missing salient entities without increasing the length. Summaries genera']

In [34]:
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10, separator=" ")
text_splitter.split_text(text)

['Selecting the “right” amount of information to include in a summary is a difficult task. \nA good',
 'A good summary should be detailed and entity-centric without being overly dense and hard to follow.',
 'to follow. To better understand this tradeoff, we solicit increasingly dense GPT-4 summaries with',
 'with what we refer to as a “Chain of Density” (CoD) prompt. Specifically, GPT-4 generates an initial',
 'an initial entity-sparse summary before iteratively incorporating missing salient entities without',
 'without increasing the length. Summaries genera']

In [35]:
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0, separator=" ")
text_splitter.split_text(text)

['Selecting the “right” amount of information to include in a summary is a difficult task. \nA good',
 'summary should be detailed and entity-centric without being overly dense and hard to follow. To',
 'better understand this tradeoff, we solicit increasingly dense GPT-4 summaries with what we refer to',
 'as a “Chain of Density” (CoD) prompt. Specifically, GPT-4 generates an initial entity-sparse summary',
 'before iteratively incorporating missing salient entities without increasing the length. Summaries',
 'genera']

In [36]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100, separator=" ")
# text 파일을 청크로 나누어줍니다.
text_splitter.split_text(text)

# document를 청크로 나누어줍니다.
split_docs = text_splitter.split_documents(docs)
len(split_docs)

8

In [37]:
split_docs[0]

Document(metadata={'source': 'https://www.bbc.com/news/business-68092814'}, page_content='Could AI \'trading bots\' transform the world of investing?Getty ImagesIt is hard for both humans and computers to predict stock market movementsSearch for "AI investing" online, and you\'ll be flooded with endless offers to let artificial intelligence manage your money.I recently spent half an hour finding out what so-called AI "trading bots" could apparently do with my investments.Many prominently suggest that they can give me lucrative returns. Yet as every reputable financial firm warns - your capital may be at risk.Or putting it more simply - you could lose your money - whether it is a human or a computer that is making stock market decisions on your behalf.Yet such has been the hype about the ability of AI over the past few years, that almost one in three investors would be happy to let a trading bot make all the decisions for them, according to one 2023 survey in the US.John Allan says inve

In [38]:
# 뉴스기사의 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
loader = WebBaseLoader(
    web_paths=("https://www.bbc.com/news/business-68092814",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "main",
            attrs={"id": ["main-content"]},
        )
    ),
)

# splitter 를 정의합니다.
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100, separator=" ")

# 문서를 로드시 바로 분할까지 수행합니다.
split_docs = loader.load_and_split(text_splitter=text_splitter)
print(f"문서의 수: {len(docs)}")
docs[0].page_content[:500]

문서의 수: 1


'Could AI \'trading bots\' transform the world of investing?Getty ImagesIt is hard for both humans and computers to predict stock market movementsSearch for "AI investing" online, and you\'ll be flooded with endless offers to let artificial intelligence manage your money.I recently spent half an hour finding out what so-called AI "trading bots" could apparently do with my investments.Many prominently suggest that they can give me lucrative returns. Yet as every reputable financial firm warns - your '

#### RecursiveTextSplitter

In [39]:
# langchain 패키지에서 RecursiveCharacterTextSplitter 클래스를 가져옵니다.
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [40]:
recursive_text_splitter = RecursiveCharacterTextSplitter(
    # 정말 작은 청크 크기를 설정합니다.
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

In [41]:
# chain of density 논문의 일부 내용을 불러옵니다
with open("data/chain-of-density.txt", "r") as f:
    text = f.read()[:500]

In [42]:
character_text_splitter = CharacterTextSplitter(
    chunk_size=100, chunk_overlap=10, separator=" "
)
for sent in character_text_splitter.split_text(text):
    print(sent)
print("===" * 20)
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, chunk_overlap=10
)
for sent in recursive_text_splitter.split_text(text):
    print(sent)

Selecting the “right” amount of information to include in a summary is a difficult task. 
A good
A good summary should be detailed and entity-centric without being overly dense and hard to follow.
to follow. To better understand this tradeoff, we solicit increasingly dense GPT-4 summaries with
with what we refer to as a “Chain of Density” (CoD) prompt. Specifically, GPT-4 generates an initial
an initial entity-sparse summary before iteratively incorporating missing salient entities without
without increasing the length. Summaries genera
Selecting the “right” amount of information to include in a summary is a difficult task.
A good summary should be detailed and entity-centric without being overly dense and hard to follow.
follow. To better understand this tradeoff, we solicit increasingly dense GPT-4 summaries with what
with what we refer to as a “Chain of Density” (CoD) prompt. Specifically, GPT-4 generates an
an initial entity-sparse summary before iteratively incorporating missing s

In [43]:
# recursive_text_splitter 에 기본 지정된 separators 를 확인합니다.
recursive_text_splitter._separators

['\n\n', '\n', ' ', '']

#### Semantic Similarity

In [44]:
# !pip install -U langchain langchain_experimental -q

In [45]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

# SemanticChunker 를 생성합니다.
semantic_text_splitter = SemanticChunker(OpenAIEmbeddings(), add_start_index=True)

In [46]:
# chain of density 논문의 일부 내용을 불러옵니다
with open("data/chain-of-density.txt", "r") as f:
    text = f.read()

for sent in semantic_text_splitter.split_text(text):
    print(sent)
    print("===" * 20)

Selecting the “right” amount of information to include in a summary is a difficult task. A good summary should be detailed and entity-centric without being overly dense and hard to follow. To better understand this tradeoff, we solicit increasingly dense GPT-4 summaries with what we refer to as a “Chain of Density” (CoD) prompt. Specifically, GPT-4 generates an initial entity-sparse summary before iteratively incorporating missing salient entities without increasing the length. Summaries generated by CoD are more abstractive, exhibit more fusion, and have less of a lead bias than GPT-4 summaries generated by a vanilla prompt. We conduct a human preference study on 100 CNN DailyMail articles and find that that humans prefer GPT-4 summaries that are more dense than those generated by a vanilla prompt and almost as dense as human written summaries. Qualitative analysis supports the notion that there exists a tradeoff between infor-mativeness and readability. 500 annotated CoD summaries, a

### 3 단계: 임베딩

In [47]:
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings

# 단계 3: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [48]:
vectorstore = FAISS.from_documents(
    documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small")
)

In [49]:
# !pip install sentence-transformers


In [50]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# 단계 3: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(
    documents=splits, embedding=HuggingFaceBgeEmbeddings()
)

c:\Users\anHye\anaconda3\envs\rag\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\anHye\.cache\huggingface\hub\models--BAAI--bge-large-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


### 4단계: 벡터스토어 생성(Create Vectorstore)

In [58]:
from langchain_community.vectorstores import FAISS

# FAISS DB 적용
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

### 5단계: Retriever 생성

#### 유사도 기반 검색

In [59]:
query = "회사의 저출생 정책이 뭐야?"

retriever = vectorstore.as_retriever(search_type="similarity")
search_result = retriever.get_relevant_documents(query)
print(search_result)

c:\Users\anHye\anaconda3\envs\rag\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content="출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책"), Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content='남성 직원의 육아휴직을 의무화한 곳도 있습니다.사내 어린이집을 밤 10시까지 운영하고 셋째를 낳으면 무조건 승진시켜 주기도 합니다.한 회사는 지난해 네쌍둥이를 낳은 직원에 의료비를 지원해 관심을 모았습니다.정부 대신 회사가 나서는 출산장려책이 사회적 분위기를 바꿀 거라는 기대가 커지는 가운데, 여력이 부족한 중소지원이 필요하다는 목소리도 나옵니다.[영상디자인 곽세미]'), Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content="[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.

In [60]:
query = "회사의 저출생 정책이 뭐야?"

retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.8}
)
search_result = retriever.get_relevant_documents(query)
print(search_result)

[Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content="출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책")]


In [61]:
query = "회사의 저출생 정책이 뭐야?"

retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 2})
search_result = retriever.get_relevant_documents(query)
print(search_result)

[Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content="출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책"), Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content='남성 직원의 육아휴직을 의무화한 곳도 있습니다.사내 어린이집을 밤 10시까지 운영하고 셋째를 낳으면 무조건 승진시켜 주기도 합니다.한 회사는 지난해 네쌍둥이를 낳은 직원에 의료비를 지원해 관심을 모았습니다.정부 대신 회사가 나서는 출산장려책이 사회적 분위기를 바꿀 거라는 기대가 커지는 가운데, 여력이 부족한 중소지원이 필요하다는 목소리도 나옵니다.[영상디자인 곽세미]')]


#### 다양한 쿼리 생성

In [62]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

query = "회사의 저출생 정책이 뭐야?"

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)

In [63]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [64]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 부영그룹이 채택한 출산 장려 정책은 무엇인가요?', '2. 출산율 증가를 위해 부영그룹이 시행한 정책은 무엇인가요?', '3. 부영그룹의 출산 지원 정책은 어떻게 운영되고 있나요?']


3

#### Ensemble Retriever

In [65]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [66]:
doc_list = [
    "난 오늘 많이 먹어서 배가 정말 부르다",
    "떠나는 저 배가 오늘 마지막 배인가요?",
    "내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.",
]

# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_texts(doc_list)
bm25_retriever.k = 2

faiss_vectorstore = FAISS.from_texts(doc_list, OpenAIEmbeddings())
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [67]:
def pretty_print(docs):
    for i, doc in enumerate(docs):
        print(f"[{i+1}] {doc.page_content}")

In [68]:
sample_query = "나 요즘 배에 정말 살이 많이 쪘어..."
print(f"[Query]\n{sample_query}\n")
relevant_docs = bm25_retriever.get_relevant_documents(sample_query)
print("[BM25 Retriever]")
pretty_print(relevant_docs)
print("===" * 20)
relevant_docs = faiss_retriever.get_relevant_documents(sample_query)
print("[FAISS Retriever]")
pretty_print(relevant_docs)
print("===" * 20)
relevant_docs = ensemble_retriever.get_relevant_documents(sample_query)
print("[Ensemble Retriever]")
pretty_print(relevant_docs)

[Query]
나 요즘 배에 정말 살이 많이 쪘어...

[BM25 Retriever]
[1] 난 오늘 많이 먹어서 배가 정말 부르다
[2] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[FAISS Retriever]
[1] 난 오늘 많이 먹어서 배가 정말 부르다
[2] 떠나는 저 배가 오늘 마지막 배인가요?
[Ensemble Retriever]
[1] 난 오늘 많이 먹어서 배가 정말 부르다
[2] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[3] 떠나는 저 배가 오늘 마지막 배인가요?


In [69]:
sample_query = "바다 위에 떠다니는 배들이 많다"
print(f"[Query]\n{sample_query}\n")
relevant_docs = bm25_retriever.get_relevant_documents(sample_query)
print("[BM25 Retriever]")
pretty_print(relevant_docs)
print("===" * 20)
relevant_docs = faiss_retriever.get_relevant_documents(sample_query)
print("[FAISS Retriever]")
pretty_print(relevant_docs)
print("===" * 20)
relevant_docs = ensemble_retriever.get_relevant_documents(sample_query)
print("[Ensemble Retriever]")
pretty_print(relevant_docs)

[Query]
바다 위에 떠다니는 배들이 많다

[BM25 Retriever]
[1] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[2] 떠나는 저 배가 오늘 마지막 배인가요?
[FAISS Retriever]
[1] 난 오늘 많이 먹어서 배가 정말 부르다
[2] 떠나는 저 배가 오늘 마지막 배인가요?
[Ensemble Retriever]
[1] 떠나는 저 배가 오늘 마지막 배인가요?
[2] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[3] 난 오늘 많이 먹어서 배가 정말 부르다


In [70]:
sample_query = "ships"
print(f"[Query]\n{sample_query}\n")
relevant_docs = bm25_retriever.get_relevant_documents(sample_query)
print("[BM25 Retriever]")
pretty_print(relevant_docs)
print("===" * 20)
relevant_docs = faiss_retriever.get_relevant_documents(sample_query)
print("[FAISS Retriever]")
pretty_print(relevant_docs)
print("===" * 20)
relevant_docs = ensemble_retriever.get_relevant_documents(sample_query)
print("[Ensemble Retriever]")
pretty_print(relevant_docs)

[Query]
ships

[BM25 Retriever]
[1] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[2] 떠나는 저 배가 오늘 마지막 배인가요?
[FAISS Retriever]
[1] 떠나는 저 배가 오늘 마지막 배인가요?
[2] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[Ensemble Retriever]
[1] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[2] 떠나는 저 배가 오늘 마지막 배인가요?


In [71]:
sample_query = "pear"
print(f"[Query]\n{sample_query}\n")
relevant_docs = bm25_retriever.get_relevant_documents(sample_query)
print("[BM25 Retriever]")
pretty_print(relevant_docs)
print("===" * 20)
relevant_docs = faiss_retriever.get_relevant_documents(sample_query)
print("[FAISS Retriever]")
pretty_print(relevant_docs)
print("===" * 20)
relevant_docs = ensemble_retriever.get_relevant_documents(sample_query)
print("[Ensemble Retriever]")
pretty_print(relevant_docs)

[Query]
pear

[BM25 Retriever]
[1] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[2] 떠나는 저 배가 오늘 마지막 배인가요?
[FAISS Retriever]
[1] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[2] 떠나는 저 배가 오늘 마지막 배인가요?
[Ensemble Retriever]
[1] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[2] 떠나는 저 배가 오늘 마지막 배인가요?


### 6단계: 프롬프트 생성(Create Prompt)

In [72]:
from langchain import hub

In [73]:
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

### 7단계: 언어모델 생성(Create LLM)

In [74]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

In [75]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    result = model.invoke("대한민국의 수도는 어디인가요?")
print(cb)

Tokens Used: 39
	Prompt Tokens: 24
	Completion Tokens: 15
Successful Requests: 1
Total Cost (USD): $3.45e-05


### RAG 템플릿 실험

In [88]:
# 단계 1: 문서 로드(Load Documents)

# 문서를 로드하고, 청크로 나누고, 인덱싱합니다.
# from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
file_path = "data/1568)누구나 한번쯤 읽어야 할 사서삼경 (미리내공방) .pdf"
# loader = PyPDFLoader(file_path=file_path)
loader = CustomPDFLoader("data/1568)누구나 한번쯤 읽어야 할 사서삼경 (미리내공방) .pdf")

docs = loader.load()
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

split_docs = text_splitter.split_documents(docs)

# 단계 3, 4: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 단계 5: 리트리버 생성(Create Retriever)
# 사용자의 질문(query) 에 부합하는 문서를 검색합니다.

# 유사도 높은 K 개의 문서를 검색합니다.
k = 3

# (Sparse) bm25 retriever and (Dense) faiss retriever 를 초기화 합니다.
bm25_retriever = BM25Retriever.from_documents(split_docs)
bm25_retriever.k = k

faiss_vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": k})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = hub.pull("rlm/rag-prompt")

# 단계 7: 언어모델 생성(Create LLM)
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


# 단계 8: 체인 생성(Create Chain)
rag_chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 결과 출력
print(f"PDF Path: {file_path}")
print(f"문서의 수: {len(docs)}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

PDF Path: data/1568)누구나 한번쯤 읽어야 할 사서삼경 (미리내공방) .pdf
문서의 수: 353
[HUMAN]
사서삼경은 어떻게 구성되어 있지?

[AI]
부영그룹은 출산을 장려하기 위해 2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다. 해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다. 또한, 셋째까지 낳는 경우에는 국민주택을 제공할 예정입니다.


In [89]:
# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "사서삼경은 어떻게 구성되어 있지?"
response = rag_chain.invoke(question)
print(response)

사서삼경은 7권의 책으로 구성되어 있으며, 사서는 《논어》, 《맹자》, 《중용》, 《대학》을 포함하고 삼경은 《시경》, 《서경》, 《역경》을 포함한다. 이들은 인간의 지침서로서 중요한 가르침을 담고 있다. 사서삼경은 동양인의 지침서로 넘어 인간의 지침서로까지 폭넓게 이해되고 있다.


In [90]:
# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "논어의 주요 내용이 뭐지?"
response = rag_chain.invoke(question)
print(response)

《논어》는 유가(儒家)의 성전(聖典)으로, 공자가 생전에 제자들과 교감하면서 드러낸 사상과 언행이 압축적으로 수록되어 있다. 이 책은 총 20편으로 구성되어 있고, 문장은 간결하고 압축적이며 철학적이다. 논어의 내용은 주로 사상적인 내용이며, 형이상학적인 측면이 강조된다.


In [91]:
# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "대학에서 수신제가 치국평천하의 뜻에 대해 설명해줘"
response = rag_chain.invoke(question)
print(response)

《대학》은 옛날의 대학에서 사람들을 가르치는 법에 대하여 서술한 책이다. 대학의 도(목적)는 밝은 덕을 밝히는 데 있으며, 백성을 새롭게 하는 데 있고, 가장 훌륭한 선에 머무르는 데 있다. 이 책은 인간을 교육하는 도를 해설한 것으로, 다른 경전의 문구는 이 책을 기본으로 한 풀이로 생각된다.
